# word2vec implementation

* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)
* [word2vec Explained: deriving Mikolov et al.'s negative-sampling word-embedding method](https://arxiv.org/abs/1402.3722)

## Overview

The original paper proposed using:
1. Matmul to extract word vectors from ```Win```.
2. Softmax to calculate scores with all the word vectors from ```Wout```.

<img src="image/word2vec_cbow_mechanism.png" align="left"/>

## Negative Sampling

### Motivation

The original implementation was computationally expensive.

1. Matmal to extract word vectors from ```Win```.
2. Softmax can happen vocabulary size times with ```Wout```.

<img src="image/word2vec_negative_sampling_motivation.png" align="left"/>


### Solution

1. Use index to extract word vectors from Win.
2. Instead of calculating softmax with all the word vectors from ```Wout```, sample small number (SL) of negative/false word vectors from ```Wout``` and calculate logistic log loss with each sample. 

<img src="image/wors2vec_neg_sample_backprop.png" align="left"/>

## Using only one Word vector space W

There is no reasoning nor proof why two word vector space are required. In the end, we only use one word vector space, which appears to be ```Win```. Use only one word vector space ```W``` to test if it works.

<img src="image/word2vec_backprop_Be.png" align="left"/>

---
# Setups

In [1]:
import cProfile
import sys
import os
import time
import re
from itertools import islice
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

## Setup for Google Colab environment

Colab gets disconnected within approx 20 min. Hence not suitable for training (or need to upgrade to the pro version).

### Clone github to Google Drive

In [2]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/drive/MyDrive/github
    !mkdir -p /content/drive/MyDrive/github
    !git clone https://github.com/oonisim/python-programs.git /content/drive/MyDrive/github


### Clone github to local directory

In [3]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/github
    !mkdir -p /content/github
    !git clone https://github.com/oonisim/python-programs.git /content/github
        
    import sys
    sys.path.append('/content/github/nlp/src')

# Jupyter notebook setups

Auto reolaod causes an error in Jupyter notebooks. Restart the Jupyter kernel for the error:
```TypeError: super(type, obj): obj must be an instance or subtype of type```
See
- https://stackoverflow.com/a/52927102/4281353
- http://thomas-cokelaer.info/blog/2011/09/382/

> The problem resides in the mechanism of reloading modules.
> Reloading a module often changes the internal object in memory which
> makes the isinstance test of super return False.

In [4]:
%load_ext line_profiler
%load_ext autoreload

## Utilites

In [5]:
%autoreload 2

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

import function.fileio as fileio
import function.text as text

---
# Data Types


In [6]:
from common.constant import (
    TYPE_INT,
    TYPE_FLOAT,
    TYPE_LABEL,
    TYPE_TENSOR,
)

# Constants

In [7]:
USE_PTB = True
DEBUG = False
VALIDATION = True

TARGET_SIZE = 1   # Size of the target event (word)
CONTEXT_SIZE = 10  # Size of the context.
WINDOW_SIZE = TARGET_SIZE + CONTEXT_SIZE
SAMPLE_SIZE = 5   # Size of the negative samples
VECTOR_SIZE = 100  # Number of features in the event vector.
STATE_FILE = "/home/oonisim/home/repository/git/oonisim/python_programs/nlp/models/word2vec_vecsize_100.pkl"

---

# Data
## Corpus

In [8]:
corpus = "To be, or not to be, that is the question that matters"
_file = "ptb.train.txt"
if USE_PTB:
    if not fileio.Function.is_file(f"~/.keras/datasets/{_file}"):
        path_to_ptb = tf.keras.utils.get_file(
            _file, 
            f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/{_file}'
        )
    corpus = fileio.Function.read_file(path_to_ptb)

In [9]:
examples = corpus.split('\n')[:5]
for line in examples:
    print(line)

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 


---
# Event (word) indexing
Index the events that have occurred in the event sequence.

In [10]:
%autoreload 2
from layer.preprocessing import (
    EventIndexing, 
)

In [11]:
word_indexing = EventIndexing(
    name="word_indexing_on_ptb",
    corpus=corpus,
    min_sequence_length=WINDOW_SIZE
)
del corpus

## EventIndexing  for the corpus

Adapt to the ```corpus``` and provides:
* event_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [12]:
words = word_indexing.list_events(range(10))
print(f"EventIndexing.vocabulary[10]:\n{words}\n")

indices = word_indexing.list_indices(words)
print(f"EventIndexing.event_to_index[10]:")
for item in zip(words, indices):
    print(item)

probabilities = word_indexing.list_probabilities(words)
print(f"\nEventIndexing.probabilities[10]:")
for word, p in zip(words, probabilities):
    print(f"{word:20s} : {p:.5e}")

EventIndexing.vocabulary[10]:
['<nil>' '<unk>' 'aer' 'banknote' 'berlitz' 'calloway' 'centrust' 'cluett' 'fromstein' 'gitano']

EventIndexing.event_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('aer', 2)
('banknote', 3)
('berlitz', 4)
('calloway', 5)
('centrust', 6)
('cluett', 7)
('fromstein', 8)
('gitano', 9)

EventIndexing.probabilities[10]:
<nil>                : 0.00000e+00
<unk>                : 1.65308e-02
aer                  : 5.34860e-06
banknote             : 5.34860e-06
berlitz              : 5.34860e-06
calloway             : 5.34860e-06
centrust             : 5.34860e-06
cluett               : 5.34860e-06
fromstein            : 5.34860e-06
gitano               : 5.34860e-06


## Sampling using the probability

Sample events according to their probabilities.

In [13]:
sample = word_indexing.sample(size=5)
print(sample)

['leaders' 'taipei' 'stunning' 'grim' 'potatoes']


## Negative Sampling
Sample events not including those events already sampled.

In [14]:
negative_indices = word_indexing.negative_sample_indices(
    size=5, excludes=word_indexing.list_indices(sample)
)
print(f"negative_indices={negative_indices} \nevents={word_indexing.list_events(negative_indices)}")

negative_indices=[9348, 6949, 2629, 2318, 2289] 
events=['wonderful' 'charitable' 'heating' 'feeling' 'film']


## Sentence to Sequence

In [15]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> 
with even brief exposures to it causing symptoms that show up decades later researchers said
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

Sentence is empty. Skipping...
Sentence is empty. Skipping...


the             :    34
asbestos        :    63
fiber           :    86
<unk>           :     1
is              :    42
unusually       :    87
<unk>           :     1
once            :    64
it              :    80
enters          :    88
the             :    34
<unk>           :     1

with           :     0
even            :     0
brief           :     0


---
# EventContext

EventContext layer generates ```(event, context)``` pairs from sequences.

In [16]:
%autoreload 2
from layer.preprocessing import (
    EventContext
)

In [17]:
event_context = EventContext(
    name="ev",
    window_size=WINDOW_SIZE,
    event_size=TARGET_SIZE
)

## Context of an event (word) in a sentence

In the sentence ```"a form of asbestos once used to make kent cigarette filters"```, one of the context windows ```a form of asbestos once``` of size 5 and event size 1 has.
* ```of``` as a target word.
* ```(a, form) and (asbestos, once)``` as its context.

### Sequence of the word indices for the sentence

In [18]:
sentences = """
a form of asbestos once used to make kent cigarette filters

N years old and former chairman of consolidated gold fields plc was named a nonexecutive director
"""

sequence = word_indexing.function(sentences)
sequence

Sentence is empty. Skipping...
Sentence is empty. Skipping...
Sentence is empty. Skipping...


array([[37, 62, 44, 63, 64, 65, 66, 67, 68, 69, 70,  0,  0,  0,  0,  0],
       [29, 30, 31, 50, 51, 43, 44, 52, 53, 54, 55, 56, 57, 37, 38, 39]])

## (event, context) pairs

For each word (event) in the setence ```(of, asbestos, ... , kent)``` excludnig the ends of the sentence, create ```(target, context)``` as:

```
[
  [of, a, form, asbestos, once],              # target is 'of', context is (a, form, asbestos, once)
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ...
]
```

### Format of the (event, context) pairs

* **E** is the target event indices
* **C** is the context indices

<img src="image/event_context_format.png" align="left"/>

In [19]:
event_context_pairs = event_context.function(sequence)
print(
    f"Event context pairs. Shape %s, Target event size %s, Window size %s." % 
    (event_context_pairs.shape, event_context.event_size, event_context.window_size)
)
event_context_pairs[:10]

Event context pairs. Shape (12, 11), Target event size 1, Window size 11.


array([[65, 37, 62, 44, 63, 64, 66, 67, 68, 69, 70],
       [66, 62, 44, 63, 64, 65, 67, 68, 69, 70,  0],
       [67, 44, 63, 64, 65, 66, 68, 69, 70,  0,  0],
       [68, 63, 64, 65, 66, 67, 69, 70,  0,  0,  0],
       [69, 64, 65, 66, 67, 68, 70,  0,  0,  0,  0],
       [70, 65, 66, 67, 68, 69,  0,  0,  0,  0,  0],
       [43, 29, 30, 31, 50, 51, 44, 52, 53, 54, 55],
       [44, 30, 31, 50, 51, 43, 52, 53, 54, 55, 56],
       [52, 31, 50, 51, 43, 44, 53, 54, 55, 56, 57],
       [53, 50, 51, 43, 44, 52, 54, 55, 56, 57, 37]], dtype=int32)

### (event, context) pairs in textual words

In [20]:
word_indexing.sequence_to_sentence(event_context_pairs[:10])

[['used',
  'a',
  'form',
  'of',
  'asbestos',
  'once',
  'to',
  'make',
  'kent',
  'cigarette',
  'filters'],
 ['to',
  'form',
  'of',
  'asbestos',
  'once',
  'used',
  'make',
  'kent',
  'cigarette',
  'filters',
  '<nil>'],
 ['make',
  'of',
  'asbestos',
  'once',
  'used',
  'to',
  'kent',
  'cigarette',
  'filters',
  '<nil>',
  '<nil>'],
 ['kent',
  'asbestos',
  'once',
  'used',
  'to',
  'make',
  'cigarette',
  'filters',
  '<nil>',
  '<nil>',
  '<nil>'],
 ['cigarette',
  'once',
  'used',
  'to',
  'make',
  'kent',
  'filters',
  '<nil>',
  '<nil>',
  '<nil>',
  '<nil>'],
 ['filters',
  'used',
  'to',
  'make',
  'kent',
  'cigarette',
  '<nil>',
  '<nil>',
  '<nil>',
  '<nil>',
  '<nil>'],
 ['chairman',
  'n',
  'years',
  'old',
  'and',
  'former',
  'of',
  'consolidated',
  'gold',
  'fields',
  'plc'],
 ['of',
  'years',
  'old',
  'and',
  'former',
  'chairman',
  'consolidated',
  'gold',
  'fields',
  'plc',
  'was'],
 ['consolidated',
  'old',
  'and'

---
# Word Embedding

Embedding is to train the model to group similar events in a close proximity in the event vector space. If two events e.g. 'pencil' and 'pen' are similar concepts, then their event vectors resides in a close distance in the event space. 

* [Thought Vectors](https://wiki.pathmind.com/thought-vectors)

## Training process

1. Calculate ```Bc```, the BoW (Bag of Words) from context event vectors.
2. Calculate ```Be```,  the BoW (Bag of Words) from target event vectors.
3. The dot product ```Ye = dot(Bc, Be)``` is given the label 1 to get them closer.
4. For each negative sample ```Ws(s)```, the dot product with ```Ys = dot(Be, Ws(s)``` is given the label 0 to get them apart. 
5. ```np.c_[Ye, Ys]``` is fowarded to the logistic log loss layer.

<img src="image/word2vec_backprop_Be.png" align="left"/>


In [21]:
%autoreload 2
from layer import (
    Embedding
)
from optimizer import (
    SGD
)

In [22]:
embedding: Embedding = Embedding(
    name="embedding",
    num_nodes=WINDOW_SIZE,
    target_size=TARGET_SIZE,
    context_size=CONTEXT_SIZE,
    negative_sample_size=SAMPLE_SIZE,
    event_vector_size=VECTOR_SIZE,
    optimizer=SGD(lr=TYPE_FLOAT(0.5)),
    dictionary=word_indexing
)

### Scores ```np.c_[Ye, Ys]``` from the Embedding

The 0th column is the scores for ```dot(Bc, Be``` for positive labels. The rest are the scores for ```dot(Bc, Ws)``` for negative labels.

In [23]:
scores = embedding.function(event_context_pairs)
print(f"Scores:\n{scores[:5]}\n")
print(f"Scores for dot(Bc, Be): \n{scores[:5, :1]}")

Scores:
[[-4.7106897e-03 -4.8856679e-03 -1.2798817e-04  1.1684952e-03 -1.6764110e-03 -1.0835825e-03]
 [-4.5479019e-03 -2.2900996e-03 -4.6602665e-03 -5.1962025e-04 -1.5752653e-03  1.7776270e-05]
 [ 6.1073392e-03  5.2285870e-04  4.1223299e-03 -1.7459705e-03 -1.3933808e-04 -5.5324985e-05]
 [-2.0977100e-03  7.4077584e-04 -1.8932857e-04 -1.4614742e-03  1.9989037e-03 -2.5731474e-03]
 [-3.3790807e-03 -5.3184214e-03 -4.6287337e-04 -2.9225873e-03 -5.5602994e-03 -1.5333435e-03]]

Scores for dot(Bc, Be): 
[[-0.00471069]
 [-0.0045479 ]
 [ 0.00610734]
 [-0.00209771]
 [-0.00337908]]


In [24]:
tf.constant([1,2,3]).numpy().ravel()

array([1, 2, 3], dtype=int32)

---
# Logistic Log Loss

Train the model to get:
1. BoW of the context event vectors close to the target event vector. Label 1
2. BoW of the context event vectors away from each of the negative sample event vectors Label 0.

This is a binary logistic classification, hence use Logistic Log Loss as the network objective function.

In [25]:
from common.function import (
    sigmoid_cross_entropy_log_loss,
    sigmoid
)
from layer.objective import (
    CrossEntropyLogLoss
)

In [26]:
loss = CrossEntropyLogLoss(
    name="loss",
    num_nodes=1,  # Logistic log loss
    log_loss_function=sigmoid_cross_entropy_log_loss
)

---
# Adapter

The logistic log loss layer expects the input of shape ```(N,M=1)```, however Embedding outputs ```(N,(1+SL)``` where ```SL``` is SAMPLE_SIZE. The ```Adapter``` layer bridges between Embedding and the Loss layers.


In [27]:
from layer.adapter import (
    Adapter
)

In [28]:
adapter_function = embedding.adapt_function_to_logistic_log_loss(loss=loss)
adapter_gradient = embedding.adapt_gradient_to_logistic_log_loss()

adapter: Adapter = Adapter(
    name="adapter",
    num_nodes=1,    # Number of output M=1 
    function=adapter_function,
    gradient=adapter_gradient
)

---
# Word2vec Network

## Construct a sequential network

$ \text {Sentences} \rightarrow EventIndexing \rightarrow EventContext \rightarrow  Embedding \rightarrow Adapter \rightarrow LogisticLogLoss$

In [29]:
from network.sequential import (
    SequentialNetwork
)

In [30]:
network = SequentialNetwork(
    name="word2vec",
    num_nodes=1,
    inference_layers=[
        word_indexing,
        event_context,
        embedding,
        adapter
    ],
    objective_layers=[
        loss
    ]
)

## Run training

### Load the saved model file if exists

In [31]:
if fileio.Function.is_file(STATE_FILE):
    print("Loading model...\nSTATE_FILE: %s" % STATE_FILE)
    state = embedding.load(STATE_FILE)

    fmt="""Model loaded.
    event_size %s
    context_size: %s
    event_vector_size: %s
    """
    print(fmt % (
        state["target_size"], 
        state["context_size"], 
        state["event_vector_size"]
    ))
else:
    print("State file does not exist. Saving the initial model.")
    embedding.save(STATE_FILE)

Loading model...
STATE_FILE: /home/oonisim/home/repository/git/oonisim/python_programs/nlp/models/word2vec_vecsize_100.pkl
Model loaded.
    event_size 1
    context_size: 10
    event_vector_size: 100
    


### Continue training

In [ ]:
NUM_SENTENCES = 50
MAX_ITERATIONS = 1000000

def sentences_generator(path_to_file, num_sentences):
    stream = fileio.Function.file_line_stream(path_to_file)
    try:
        while True:
            _lines = fileio.Function.take(num_sentences, stream)
            yield np.array(_lines)
    finally:
        stream.close()

# Sentences for the trainig
path_to_input = path_to_ptb
source = sentences_generator(
    path_to_file=path_to_input, num_sentences=NUM_SENTENCES
)

# Restore the state if exists.
state = embedding.load(STATE_FILE)

# Continue training
profiler = cProfile.Profile()
profiler.enable()

total_sentences = 0
epochs = 0

for i in range(MAX_ITERATIONS):
    try:
        sentences = next(source)
        total_sentences += len(sentences)

        start = time.time()
        network.train(X=sentences, T=np.array([0]))

        if i % 100 == 0:
            print(
                f"Batch {i:05d} of {NUM_SENTENCES} sentences: "
                f"Average Loss: {np.mean(network.history[-1]):10f} "
                f"Duration {time.time() - start:3f}"
            )
        if i % 10 == 0:
            embedding.save(STATE_FILE)
        
    except fileio.Function.GenearatorHasNoMore as e:
        # Next epoch
        print(f"epoch {epochs} batches {i:05d} done")
        print(f"Average loss: {np.mean(network.history):15f}")
        epochs += 1
        source.close()
        source = sentences_generator(
            path_to_file=path_to_input, num_sentences=NUM_SENTENCES
        )

    except Exception as e:
        print("Unexpected error:", sys.exc_info()[0])
        source.close()
        raise e

embedding.save(STATE_FILE)

profiler.disable()
profiler.print_stats(sort="cumtime")

Batch 00000 of 50 sentences: Average Loss:   0.176311 Duration 1.399338
Batch 00100 of 50 sentences: Average Loss:   0.038964 Duration 1.436963
Batch 00200 of 50 sentences: Average Loss:   0.039094 Duration 1.287315
Batch 00300 of 50 sentences: Average Loss:   0.038099 Duration 1.330058


---
# Evaluate the vector space

Verify if the trained model, or the vector space W, has encoded the words in a way that **similar** words are close in the vector space.

* [How to measure the similarity among vectors](https://math.stackexchange.com/questions/4132458)

In [ ]:
n = 10
context = "smoking".split()
word_indices = np.array(word_indexing.list_indices(context), dtype=TYPE_INT)

print(f"Words {context}")
print(f"Word indices {word_indices}")
print(f"prediction for {context}:\n{word_indexing.list_events([embedding.predict(word_indices, n)])}")

In [ ]:
np.zeros(shape=(141, 11))